In [1]:
import sys
import time
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
from context import Context
from tools.runners.account_state_runner import get_account_keys_online
from tools.runners.account_state_runner import report_key_files_compare
from contracts.contract_identities import DEXContractInterface

context = Context()

2024-04-17 17:45:51,966 - WARNING - Structure definition for egld_wraps not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json
2024-04-17 17:45:51,968 - WARNING - Structure definition for pairs_view not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json
2024-04-17 17:45:51,970 - WARNING - Structure definition for stakings_boosted not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json
2024-04-17 17:45:51,971 - WARNING - Structure definition for metastakings_boosted not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json
2024-04-17 17:45:51,972 - WARNING - Structure definition for position_creator not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json


loaded 10 accounts from 1 PEM files.


2024-04-17 17:45:54,863 - DEBUG - Account.sync_nonce() done: 3552
2024-04-17 17:45:55,042 - DEBUG - Account.sync_nonce() done: 3552
2024-04-17 17:45:55,044 - WARNING - Loaded 11 tokens instead of expected 10.
2024-04-17 17:45:55,228 - DEBUG - Account.sync_nonce() done: 3552
2024-04-17 17:45:55,230 - INFO - Loaded 0 egld_wraps.
2024-04-17 17:45:55,231 - INFO - Loaded 1 locked_assets.
2024-04-17 17:45:55,232 - INFO - Loaded 1 proxies.
2024-04-17 17:45:55,233 - INFO - Loaded 0 simple_locks.
2024-04-17 17:45:55,234 - INFO - Loaded 1 simple_locks_energy.
2024-04-17 17:45:55,236 - INFO - Loaded 1 fees_collectors.
2024-04-17 17:45:55,237 - INFO - Loaded 1 unstakers.
2024-04-17 17:45:55,239 - INFO - Loaded 1 proxies_v2.
2024-04-17 17:45:55,240 - INFO - Loaded 0 router.
2024-04-17 17:45:55,241 - INFO - Loaded 1 router_v2.
2024-04-17 17:45:55,242 - INFO - Loaded 0 pairs.
2024-04-17 17:45:55,244 - INFO - Loaded 9 pairs_v2.
2024-04-17 17:45:55,244 - INFO - Loaded 0 pairs_view.
2024-04-17 17:45:55,

In [2]:
# wrap the upgrade function of each contract around a state compare procedure
def upgrade_contract_with_check(contract: DEXContractInterface, *args):
    try:
        # get the current state of the account
        get_account_keys_online(contract.address, config.DEFAULT_PROXY, with_save_in="state_dump/pre_contract_state.json")
    except Exception as e:
        print(f"Error getting the account state: {e}")
        # ask for user accept to continue or cancel
        if input("Continue without state? (y/n): ") != "y":
            return

    # upgrade the contract
    contract.contract_upgrade(*args)

    try:
        # get the new state of the account
        get_account_keys_online(contract.address, config.DEFAULT_PROXY, with_save_in="state_dump/post_contract_state.json")
        # compare the states
        report_key_files_compare("state_dump", "pre", "post", True)
    except Exception as e:
        print(f"Error getting the account state: {e}")

Energy contract setup

In [10]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

In [ ]:
energy_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.SIMPLE_LOCK_ENERGY_BYTECODE_PATH, [], True)

In [14]:
upgrade_contract_with_check(energy_contract, context.deployer_account, context.network_provider.proxy, config.SIMPLE_LOCK_ENERGY_BYTECODE_PATH, [], True)

Error getting the account state: Url = [https://proxy-shadowfork-three.elrond.ro/address/erd1qqqqqqqqqqqqqpgq0tajepcazernwt74820t8ef7t28vjfgukp2sw239f3/keys], error = {'data': None, 'error': 'get key value pairs error: sending request error', 'code': 'internal_issue'}


2024-04-17 17:26:55,424 - INFO - Upgrade simple lock energy contract
2024-04-17 17:26:55,425 - DEBUG - Upgrade SimpleLockEnergyContract contract
2024-04-17 17:26:56,535 - DEBUG - Upgrade arguments: []
2024-04-17 17:26:57,704 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/bed859079aadee416d14c5963ab4771fa400b964d96972537624c9a0a9761251


Error getting the account state: Url = [https://proxy-shadowfork-three.elrond.ro/address/erd1qqqqqqqqqqqqqpgq0tajepcazernwt74820t8ef7t28vjfgukp2sw239f3/keys], error = {'data': None, 'error': 'get key value pairs error: sending request error', 'code': 'internal_issue'}


Proxy dex setup

In [3]:
from contracts.dex_proxy_contract import DexProxyContract
from utils.utils_chain import WrapperAddress as Address, hex_to_string
from utils.contract_data_fetchers import ProxyContractDataFetcher

dex_proxy_contract: DexProxyContract
dex_proxy_contract = context.get_contracts(config.PROXIES_V2)[0]

data_fetcher = ProxyContractDataFetcher(Address(dex_proxy_contract.address), context.network_provider.proxy.url)
old_locked_token = hex_to_string(data_fetcher.get_data("getLockedTokenIds")[0])
old_factory_address = context.get_contracts(config.LOCKED_ASSETS)[0].address
print(f"old_locked_token: {old_locked_token}")
print(f"old_factory_address: {old_factory_address}")

old_locked_token: LKMEX-aab910
old_factory_address: erd1qqqqqqqqqqqqqpgqjpt0qqgsrdhp2xqygpjtfrpwf76f9nvg2jpsg4q7th


In [20]:
dex_proxy_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.PROXY_V2_BYTECODE_PATH, [old_locked_token, old_factory_address])

2024-04-17 17:45:36,071 - INFO - upgrade DexProxyContract contract
2024-04-17 17:45:36,072 - ERROR - Failed to upgrade DexProxyContract contract due to unexpected number of arguments received!
2024-04-17 17:45:36,073 - DEBUG - Unexpected arguments: ['LKMEX-aab910', 'erd1qqqqqqqqqqqqqpgqjpt0qqgsrdhp2xqygpjtfrpwf76f9nvg2jpsg4q7th']


old_locked_token: LKMEX-aab910
old_factory_address: erd1qqqqqqqqqqqqqpgqjpt0qqgsrdhp2xqygpjtfrpwf76f9nvg2jpsg4q7th


('', '')

In [17]:
upgrade_contract_with_check(dex_proxy_contract, context.deployer_account, context.network_provider.proxy, config.PROXY_V2_BYTECODE_PATH, [])

Error getting the account state: Url = [https://proxy-shadowfork-three.elrond.ro/address/erd1qqqqqqqqqqqqqpgqt6ltx52ukss9d2qag2k67at28a36xc9lkp2sr06394/keys], error = {'data': None, 'error': 'get key value pairs error: sending request error', 'code': 'internal_issue'}


2024-04-17 17:29:33,542 - INFO - upgrade DexProxyContract contract
2024-04-17 17:29:33,543 - DEBUG - Upgrade DexProxyContract contract
2024-04-17 17:29:34,654 - DEBUG - Upgrade arguments: []
2024-04-17 17:29:35,910 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/4a67cc0d2c6a9ca6ec1ebccbfa358601463c8fdb17418e60db27e8f0bbfc013d


Error getting the account state: Url = [https://proxy-shadowfork-three.elrond.ro/address/erd1qqqqqqqqqqqqqpgqt6ltx52ukss9d2qag2k67at28a36xc9lkp2sr06394/keys], error = {'data': None, 'error': 'get key value pairs error: sending request error', 'code': 'internal_issue'}


Farm contract setup

In [ ]:
from contracts.farm_contract import FarmContract
farm_contract: FarmContract
farm_contracts = context.get_contracts(config.FARMS_V2)

In [ ]:
for farm_contract in farm_contracts:
    farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.FARM_V3_BYTECODE_PATH, [], True)

In [ ]:
for farm_contract in farm_contracts:
    upgrade_contract_with_check(farm_contract, context.deployer_account, context.network_provider.proxy, config.FARM_V3_BYTECODE_PATH, [], True)

In [ ]:
# resume all
context.deployer_account.sync_nonce(context.network_provider.proxy)

for contract in farm_contracts:
    tx_hash = contract.resume(context.deployer_account, context.network_provider.proxy)

Staking contract setup

In [ ]:
from contracts.staking_contract import StakingContract
staking_contract: StakingContract
staking_contracts = context.get_contracts(config.STAKINGS_V2)

In [ ]:
for staking_contract in staking_contracts:
    staking_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.STAKING_V3_BYTECODE_PATH, [], True)

In [ ]:
for staking_contract in staking_contracts:
    upgrade_contract_with_check(staking_contract, context.deployer_account, context.network_provider.proxy, config.STAKING_V3_BYTECODE_PATH, [], True)

In [ ]:
from time import sleep
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
context.deployer_account.sync_nonce(context.network_provider.proxy)
simple_lock_energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

for contract in staking_contracts:
    tx_hash = contract.set_boosted_yields_rewards_percentage(context.deployer_account, context.network_provider.proxy, 6000)
    tx_hash = contract.set_boosted_yields_factors(context.deployer_account, context.network_provider.proxy, 
                                                  [2, 1, 0, 1, 1])
    tx_hash = contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address)
    tx_hash = simple_lock_energy_contract.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, contract.address)
    sleep(2)

In [ ]:
# resume all
context.deployer_account.sync_nonce(context.network_provider.proxy)

for contract in staking_contracts:
    tx_hash = contract.resume(context.deployer_account, context.network_provider.proxy)

Metastaking contract setup

In [ ]:
from contracts.metastaking_contract import MetaStakingContract
metastaking_contract: MetaStakingContract
metastaking_contracts = context.get_contracts(config.METASTAKINGS_V2)

In [ ]:
for metastaking_contract in metastaking_contracts:
    metastaking_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.STAKING_PROXY_V3_BYTECODE_PATH, [], True)

In [ ]:
for metastaking_contract in metastaking_contracts:
    upgrade_contract_with_check(metastaking_contract, context.deployer_account, context.network_provider.proxy, config.STAKING_PROXY_V3_BYTECODE_PATH, [], True)

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)

for contract in metastaking_contracts:
    tx_hash = contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address)

2023-12-19 12:58:36,079 - DEBUG - Account.sync_nonce() done: 3531
2023-12-19 12:58:36,080 - INFO - Set energy factory address in proxy staking contract
2023-12-19 12:58:36,081 - DEBUG - Calling setEnergyFactoryAddress at erd1qqqqqqqqqqqqqpgqjvcrk4tun6l438crr0jcz7q3yzp66smwkp2shzmazs
2023-12-19 12:58:36,081 - DEBUG - Args: ['erd1qqqqqqqqqqqqqpgq0tajepcazernwt74820t8ef7t28vjfgukp2sw239f3']
2023-12-19 12:58:36,655 - DEBUG - Contract call arguments: [erd1qqqqqqqqqqqqqpgq0tajepcazernwt74820t8ef7t28vjfgukp2sw239f3]
2023-12-19 12:58:37,509 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/628d12c699ee40548534239b9346c6799c6ef2f0812906d56c8cfed0f02a197a
2023-12-19 12:58:37,509 - INFO - Set energy factory address in proxy staking contract
2023-12-19 12:58:37,510 - DEBUG - Calling setEnergyFactoryAddress at erd1qqqqqqqqqqqqqpgqduftj7u5u5d4ql2znchcmrcet9kcze7ckp2sk4474f
2023-12-19 12:58:37,511 - DEBUG - Args: ['erd1qqqqqqqqqqqqqpgq0tajepcaze